[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb)

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running.


## Prerequisites

Install dependencies.

In [2]:
!pip install --upgrade pinecone-client

Set up Pinecone. Get your Pinecone API key [here](https://www.pinecone.io/start).

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [3]:
import os
import pinecone

# get api key from app.pinecone.io
api_key = '31d9630a-fd17-440d-bae5-a82492ec3883'
# find your environment next to the api key in pinecone console
env = 'gcp-starter'

pinecone.init(
    api_key=api_key,
    environment=env
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
!pip install --upgrade pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 2.2.2
    Uninstalling pinecone-client-2.2.2:
      Successfully uninstalled pinecone-client-2.2.2


In [4]:
import pinecone.info

version_info = pinecone.info.version()
server_version = ".".join(version_info.server.split(".")[:2])
client_version = ".".join(version_info.client.split(".")[:2])

assert client_version == server_version, "Please upgrade pinecone-client."

AssertionError: ignored

In [5]:
# Giving our index a name
index_name = "hello-pinecone"

In [6]:
# Delete the index, if an index of the same name already exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

Creating a Pinecone Index.

In [7]:
import time

dimensions = 3
pinecone.create_index(name=index_name, dimension=dimensions, metric="cosine")

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

In [8]:
index = pinecone.Index(index_name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [9]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('id').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [10]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{'upserted_count': 2}

In [11]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
index.query(
    vector=[2., 2., 2.],
    top_k=5,
    include_values=True) # returns top_k matches

{'matches': [], 'namespace': ''}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [13]:
pinecone.delete_index(index_name)